#Install libraries

In [1]:
!pip install -q google-adk google-genai python-dotenv


#Configure Gemini and sanity check

In [3]:
import os
from google import genai

os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

client = genai.Client(api_key=GEMINI_API_KEY)

resp = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents="Say hello as a software bug-fixing assistant.",
)
print(resp.text)


Hello! Bug-squashing assistant ready and reporting for duty! How can I help you fix those pesky problems today?



#Bug-analysis tool

In [5]:
from google.adk.tools import FunctionTool

def analyze_bug(code: str, error_message: str = "") -> str:
    """
    Use Gemini to analyze a code snippet and optional error message,
    and return a concise explanation + suggested fix.
    """
    prompt = (
        "You are a software bug-fixing assistant.\n\n"
        "Code:\n"
        "```python\n"
        f"{code}\n"
        "```\n\n"
        "Error message (may be empty):\n"
        f"{error_message}\n\n"
        "1. Explain likely root cause.\n"
        "2. Suggest a concrete fix.\n"
        "3. If you are not sure, say so.\n"
    )

    resp = client.models.generate_content(
        model="gemini-2.0-flash-001",
        contents=prompt,
    )
    return resp.text.strip()

bug_tool = FunctionTool(func=analyze_bug)


/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


In [6]:
sample_code = "def divide(a, b):\n    return a / b\n\nprint(divide(10, 0))"
sample_error = "ZeroDivisionError: division by zero"

print(analyze_bug(sample_code, sample_error))


Okay, I can help with that.

1.  **Likely Root Cause:**

    The error `ZeroDivisionError: division by zero` occurs because the code attempts to divide the variable `a` by the variable `b`, and in this specific case, `b` is equal to 0. Division by zero is mathematically undefined and will cause a runtime error in most programming languages, including Python.

2.  **Suggest a Concrete Fix:**

    The most common and robust fix is to add a check to ensure that `b` is not zero before performing the division. If `b` is zero, you can either return a default value (like 0, `None`, or infinity, depending on the context), raise a different exception, or print an error message. Here's an example that returns `None` if `b` is zero:

    ```python
    def divide(a, b):
        if b == 0:
            return None  # Or any other appropriate default value or error handling
        else:
            return a / b

    print(divide(10, 0))
    print(divide(10, 2))
    ```

    Another approach would be

#Bug assistant agent

In [8]:
from google.adk.agents import LlmAgent

bug_agent = LlmAgent(
    name="bug_assistant",
    model="gemini-2.0-flash-001",
    instruction=(
        "You are a software bug assistant.\n"
        "- The user will send you code snippets and possibly error messages.\n"
        "- When you need to reason about the bug, call the analyze_bug tool.\n"
        "- Then summarize the root cause and suggested fix clearly for the user."
    ),
    tools=[bug_tool],
)


#Runner and session

In [9]:
from google.adk.runners import InMemoryRunner

APP_NAME = "bug_assistant_app"
USER_ID = "demo-bug-user"

runner = InMemoryRunner(agent=bug_agent, app_name=APP_NAME)

# Colab supports top-level await
session = await runner.session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
)

print("Session ID:", session.id)


Session ID: 36bd45c3-1dc1-40bf-931c-9ff75b36afef


#Helper to talk to bug assistant

In [10]:
from google.genai.types import Content, Part

def ask_bug_agent(code: str, error_message: str = ""):
    user_text = (
        "I have this Python code:\n\n"
        "```python\n"
        f"{code}\n"
        "```\n\n"
        f"And this error:\n{error_message}\n\n"
        "Please help me understand the root cause and how to fix it."
    )

    user_msg = Content(
        role="user",
        parts=[Part.from_text(text=user_text)],
    )

    print("=== USER QUESTION ===")
    print(user_text)
    print("\n=== BUG ASSISTANT RESPONSE ===")

    for event in runner.run(
        user_id=session.user_id,
        session_id=session.id,
        new_message=user_msg,
    ):
        if hasattr(event, "is_final_response") and event.is_final_response():
            if event.content:
                for part in event.content.parts:
                    text = getattr(part, "text", None)
                    if text:
                        print(text.strip())

# try it with the same ZeroDivision example
sample_code = "def divide(a, b):\n    return a / b\n\nprint(divide(10, 0))"
sample_error = "ZeroDivisionError: division by zero"

ask_bug_agent(sample_code, sample_error)


=== USER QUESTION ===
I have this Python code:

```python
def divide(a, b):
    return a / b

print(divide(10, 0))
```

And this error:
ZeroDivisionError: division by zero

Please help me understand the root cause and how to fix it.

=== BUG ASSISTANT RESPONSE ===


The root cause is that the code attempts to divide by zero, which is mathematically undefined and results in a `ZeroDivisionError`.

To fix this, add a check to ensure that the divisor is not zero before performing the division. If it is zero, you can return a specific value (like `None` or 0), raise a `ValueError` exception, or print an error message, depending on the context.
